In [205]:
#Import all necessary packages

import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import datetime as dt
from selenium.webdriver import Chrome
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [253]:
#Start with RVezy

base_url = 'https://www.rvezy.com/rv-search?searchAddress={city},%20{prov},%20Canada'
search_cities = [('Toronto', 'ON'), ('Ottawa','ON'),('Vancouver','BC'),('Calgary', 'AB'), ('Montreal', 'QC'),('Winnipeg', 'MB')]
rows = []
xpath = "//*[@id='__layout']/div/div/div[2]/div/ul/li/button[@aria-label='Go to next page']"

for tup in search_cities: 
    city = tup[0]
    prov = tup[1]
    url = base_url.format(city = city, prov = prov)
    
    
    #Ensure Chromedriver is downloaded & located in current working directory 
    driver = Chrome(executable_path = os.getcwd()+'/chromedriver')
    driver.get(url)
    
    while True: 
        try: 
            WebDriverWait(driver, timeout = 5).until(lambda d: d.find_element_by_class_name('card-body'))
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
    
            for item in soup.find_all('div', class_='card-body'):

                price = item.find('span', class_='nightly-price').text
                name = item.find('span', class_='rv-name').text
                location = item.find('small', class_='location-type').text
                distance = item.find('small', class_='search-distance').text
                sleeps = item.find('span', class_='sleeps').text[-1]

                try: 
                    featured = item.find('span', class_='badge').text
                except: 
                    featured = None

                try: 
                    rating_value = item.output.get("aria-valuenow")
                    rating_count = item.find('span', class_='avg-rating').text
                except: 
                    rating_value = None
                    rating_count = None

                search_city = city
                marketplace = 'RVezy'
                date = dt.date.today()

                rows.append([name,price,featured,location,sleeps,rating_value,rating_count,search_city,marketplace,date])
            
            el2 = WebDriverWait(driver, timeout = 5).until(lambda d: d.find_element_by_xpath(xpath))
            el2.click()
        
        except: 
            print('timeout')
            break

columns = ['name','price','featured','location','sleeps','rating_value','rating_count','search_city','marketplace','date']
rv_df = pd.DataFrame(rows, columns = columns) 



timeout
timeout
timeout
timeout
timeout
timeout


In [393]:
rv_df.head(3)

,name,price,featured,location,sleeps,nightly_rate,currency,class,rating_value,rating_count,search_city,marketplace,date
0,Dutchmen Aspen Trail 2340 Bhs,$135 CAD/Night,Featured,Caledon-east,8,135,CAD,Travel Trailer,5.0,5,Toronto,RVezy,2021-11-02
1,Mike's 2021 Palomino Solaire 147x Hybrid,$120 CAD/Night,Featured,Whitby,7,120,CAD,Hybrid,5.0,1,Toronto,RVezy,2021-11-02
2,JD’s RV,$275 CAD/Night,Featured,Orangeville,7,275,CAD,Class A,5.0,12,Toronto,RVezy,2021-11-02


In [351]:
#Clean columns and create new columns where appropriate
rv_df['currency'] = rv_df['price'].apply(lambda x: x.split(" ")[1].split('/')[0])
rv_df['nightly_rate'] = rv_df['price'].apply(lambda x: int(x.split(" ")[0].strip("$")))
rv_df['class'] = rv_df['location'].apply(lambda x: x.replace("\n","").split("•")[1]).str.strip()
rv_df['location'] = rv_df['location'].apply(lambda x: x.replace("\n","").split("•")[0].split(",")[0]).str.strip()
rv_df['date'] = pd.to_datetime(rv_df['date'])
rv_df['rating_count'] = rv_df['rating_count'].apply(lambda x: re.search(r'(\d+)', x)[0] if x != None else 0)

#Change column types to appropriate types
rv_df[['marketplace', 'search_city', 'featured', 'currency','class']] = rv_df[['marketplace', 'search_city', 'featured', 'currency','class']].astype('category')
rv_df[['sleeps','rating_count','nightly_rate']] = rv_df[['sleeps','rating_count','nightly_rate']].astype('int')
rv_df['rating_value'] = rv_df['rating_value'].fillna(0).astype('float')

#Reorder columns 
columns = columns = ['name','price','featured','location','sleeps','nightly_rate','currency','class','rating_value','rating_count','search_city','marketplace','date']
rv_df = rv_df[columns]
rv_df.info()
rv_df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141 entries, 0 to 7140
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          7141 non-null   object        
 1   price         7141 non-null   object        
 2   featured      1799 non-null   category      
 3   location      7141 non-null   object        
 4   sleeps        7141 non-null   int64         
 5   nightly_rate  7141 non-null   int64         
 6   currency      7141 non-null   category      
 7   class         7141 non-null   category      
 8   rating_value  7141 non-null   float64       
 9   rating_count  7141 non-null   int64         
 10  search_city   7141 non-null   category      
 11  marketplace   7141 non-null   category      
 12  date          7141 non-null   datetime64[ns]
dtypes: category(5), datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 482.2+ KB


,name,price,featured,location,sleeps,nightly_rate,currency,class,rating_value,rating_count,search_city,marketplace,date
0,Dutchmen Aspen Trail 2340 Bhs,$135 CAD/Night,Featured,Caledon-east,8,135,CAD,Travel Trailer,5.0,5,Toronto,RVezy,2021-11-02
1,Mike's 2021 Palomino Solaire 147x Hybrid,$120 CAD/Night,Featured,Whitby,7,120,CAD,Hybrid,5.0,1,Toronto,RVezy,2021-11-02
2,JD’s RV,$275 CAD/Night,Featured,Orangeville,7,275,CAD,Class A,5.0,12,Toronto,RVezy,2021-11-02
3,Kevin's Coachmen Apex 259bhss Travel Trailer,$139 CAD/Night,Featured,Brantford,8,139,CAD,Travel Trailer,0.0,0,Toronto,RVezy,2021-11-02
4,Ryan's Starcraft Autumn Ridge Travel Trailer,$120 CAD/Night,Featured,Kitchener,0,120,CAD,Travel Trailer,0.0,0,Toronto,RVezy,2021-11-02


In [197]:
#Scrape Outdoorsy website with same process


base_url = 'https://ca.outdoorsy.com/rv-search?address={city}%2C%20{prov}%2C%20Canada&map=false'
url = 'https://ca.outdoorsy.com/rv-search?address=Toronto%2C%20Ontario%2C%20Canada&map=false'

search_cities = [('Toronto', 'ON'), ('Ottawa','ON'),('Vancouver','BC'),('Calgary', 'AB'), ('Montreal', 'QC'),('Winnipeg', 'MB')]

rows = []

for tup in search_cities: 
    city = tup[0]
    prov = tup[1]
    url = base_url.format(city = city, prov = prov)


    driver = Chrome(executable_path = os.getcwd()+'/chromedriver')
    driver.get(url)
    
    while True: 
        try: 
            
            el = WebDriverWait(driver, timeout = 5).until(lambda d: d.find_element_by_class_name('_CardRental_b3dfl3'))
            page = driver.page_source

            soup = BeautifulSoup(page, 'html.parser')

            items = soup.find_all('div', class_='_CardRental_b3dfl3')

            for item in items: 
                vehicle_type = item.ul.find_all('li')[0].text
                location = item.ul.find_all('li')[1].text
                name  = item.a.get('title')
                try: 
                    price = item.find_all('em')[2].text
                    length = item.find_all('em')[1].text
                    sleeps = item.find_all('em')[0].text
                except: 
                    length = item.find_all('em')[0].text
                    price = item.find_all('em')[1].text
                    sleeps = None

                try: 
                    rating_value = item.find('div', class_='_Rating_1jp9g1').get('aria-label')
                    rating_count = item.find('div', class_='_Rating_1jp9g1').span.text
                except: 
                    rating_value = None
                    rating_count = None

                search_city = city
                marketplace = 'Outdoorsy'
                date = dt.date.today()

                rows.append([name,price,location,sleeps,rating_value,rating_count,vehicle_type,length,search_city,marketplace,date])

            el2 = WebDriverWait(driver, timeout = 5).until(lambda d: d.find_element_by_class_name('_next_umq6wz'))
            el2.click()
        except: 
            break 
    driver.quit()

columns = ['name','price','location','sleeps','rating_value','rating_count','vehicle_type','length','search_city','marketplace','date']
od_df = pd.DataFrame(rows, columns = columns) 




In [333]:
#Clean columns 
od_df['nightly_rate'] = od_df['price'].str.replace("$","").str.replace(",","").astype('int')
od_df['rating_value'] = od_df['rating_value'].apply(lambda x: x.split(' ')[0] if x != None else 0).astype('float')
od_df['rating_count'] = od_df['rating_count'].apply(lambda x: re.search(r'(\d+)',x)[0] if x != None else 0).astype('int')
od_df['class'] = od_df['vehicle_type'].str.replace('\n',"").str.strip().astype('category')
od_df['location'] = od_df['location'].str.replace('\n',"").str.strip().str.title()
od_df['date'] = pd.to_datetime(od_df['date'])
od_df['sleeps'] = od_df['sleeps'].apply(lambda x: int(x) if x!= None else 0)
od_df[['search_city', 'marketplace']] = od_df[['search_city', 'marketplace']].astype('category')

#Assuming since no currency is specified it is all listed in $CAD
od_df['currency'] = 'CAD'
od_df['currency'] = od_df['currency'].astype('category')

od_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873 entries, 0 to 1872
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          1873 non-null   object        
 1   price         1873 non-null   object        
 2   location      1873 non-null   object        
 3   sleeps        1873 non-null   int64         
 4   rating_value  1873 non-null   float64       
 5   rating_count  1873 non-null   int64         
 6   vehicle_type  1873 non-null   object        
 7   length        1873 non-null   object        
 8   search_city   1873 non-null   category      
 9   marketplace   1873 non-null   category      
 10  date          1873 non-null   datetime64[ns]
 11  nightly_rate  1873 non-null   int64         
 12  class         1873 non-null   category      
 13  currency      1873 non-null   category      
dtypes: category(4), datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 154.9+ 

In [392]:
#Merge dataframes into one, output file as csv so we can visualize in Tableau

merged_df = pd.concat([rv_df,od_df])
merged_df.info()

merged_df['rate_per_sleeper'] = merged_df['nightly_rate'] / merged_df['sleeps']
mask = merged_df['rate_per_sleeper'] > 1000
merged_df.loc[mask, 'rate_per_sleeper'] = None

## Export merged_df to CSV with today's date in the title. 
cols = ['vehicle_type','length','featured']
merged_df.drop(columns = cols, inplace = True)
file_name = str(dt.date.today())+"_RVezy_Outdoorsy_Data.csv"
merged_df.to_csv(path_or_buf="Output/"+file_name)

print("Data scraped & saved on {date}".format(date=dt.date.today()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9014 entries, 0 to 1872
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          9014 non-null   object        
 1   price         9014 non-null   object        
 2   featured      1799 non-null   object        
 3   location      9014 non-null   object        
 4   sleeps        9014 non-null   int64         
 5   nightly_rate  9014 non-null   int64         
 6   currency      9014 non-null   category      
 7   class         9014 non-null   object        
 8   rating_value  9014 non-null   float64       
 9   rating_count  9014 non-null   int64         
 10  search_city   9014 non-null   category      
 11  marketplace   9014 non-null   object        
 12  date          9014 non-null   datetime64[ns]
 13  vehicle_type  1873 non-null   object        
 14  length        1873 non-null   object        
dtypes: category(2), datetime64[ns](1), flo

In [359]:
merged_df.groupby(['search_city','marketplace']).mean()

sleeps  nightly_rate  rating_value  rating_count
search_city marketplace                                                    
Calgary     Outdoorsy    5.256667    144.380000      3.652333      5.640000
            RVezy        5.235714    134.562500      4.546429      8.094643
Montreal    Outdoorsy    4.577143    171.820000      2.422571      1.802857
            RVezy        4.730826    150.439528      4.027655      6.269174
Ottawa      Outdoorsy    5.474286    180.491429      3.010286      3.160000
            RVezy        4.818750    145.352885      4.474519      7.101923
Toronto     Outdoorsy    5.726064    146.718085      3.338298      4.148936
            RVezy        5.230921    140.060526      4.338816      8.123684
Vancouver   Outdoorsy    4.488333    153.365000      3.222667      3.575000
            RVezy        5.182453    135.547358      4.046859      5.823529
Winnipeg    Outdoorsy    5.694444    174.666667      3.759722      3.125000
            RVezy        4.854839    135.790323      3.782258      4.354839

In [360]:
merged_df.groupby(['search_city','marketplace']).count()

name  location  sleeps  nightly_rate  currency  \
search_city marketplace                                                   
Calgary     Outdoorsy     300       300     300           300       300   
            RVezy        1120      1120    1120          1120      1120   
Montreal    Outdoorsy     350       350     350           350       350   
            RVezy        1356      1356    1356          1356      1356   
Ottawa      Outdoorsy     175       175     175           175       175   
            RVezy        2080      2080    2080          2080      2080   
Toronto     Outdoorsy     376       376     376           376       376   
            RVezy        1520      1520    1520          1520      1520   
Vancouver   Outdoorsy     600       600     600           600       600   
            RVezy        1003      1003    1003          1003      1003   
Winnipeg    Outdoorsy      72        72      72            72        72   
            RVezy          62        62      62            62        62   

                         class  rating_value  rating_count  date  
search_city marketplace                                           
Calgary     Outdoorsy      300           300           300   300  
            RVezy         1120          1120          1120  1120  
Montreal    Outdoorsy      350           350           350   350  
            RVezy         1356          1356          1356  1356  
Ottawa      Outdoorsy      175           175           175   175  
            RVezy         2080          2080          2080  2080  
Toronto     Outdoorsy      376           376           376   376  
            RVezy         1520          1520          1520  1520  
Vancouver   Outdoorsy      600           600           600   600  
            RVezy         1003          1003          1003  1003  
Winnipeg    Outdoorsy       72            72            72    72  
            RVezy           62            62            62    62